In [1]:
import getpass
import json
import pandas as pd
import tiktoken
import tqdm

from data_processing import pers_labels
from openai import OpenAI
from sklearn.metrics import f1_score, precision_score, recall_score

In [6]:
client = OpenAI(api_key=getpass.getpass("please enter your openai api key"))

In [81]:
PWD = os.environ["WORKSPACE_PATH"]

data_type = "big 5_1_top_lbl"

label_mode = pers_labels.MBPT if pers_labels.MBPT.lower() in data_type else pers_labels.BIG_5

index = [idx for idx in range(5) if str(idx) in data_type][0]

pers_defs = {
    pers_labels.MBPT: {
        0: (("I", "introverted"), ("E", "extroverted")),
        1: (("S", "sensing"), ("N", "intuitive")),
        2: (("F", "feeling"), ("T", "thinking")),
        3: (("J", "judging"), ("P", "perceiving")),
    },
    pers_labels.BIG_5: {
        0: (("S", "social"), ("R", "reserved")),
        1: (("L", "limbic"), ("C", "calm")),
        2: (("O", "organized"), ("U", "unstructured")),
        3: (("A", "agreeable"), ("E", "egocentric")),
        4: (("N", "non-curious"), ("I", "inquisitive")),
    }
}

((label1, label1_def), (label2, label2_def)) = pers_defs[label_mode][index]

## loading data

In [82]:
with open(f"{PWD}/data/cornell_movies/speakers.json", "r+") as fp:
    fp_parsed = json.load(fp)
    chars_meta = {}
    chars_meta_rows = []
    for char in fp_parsed:
        meta = fp_parsed[char]["meta"]
        meta["character_name"] = meta["character_name"].lower()
        meta["char_id"] = char
        chars_meta[char] = meta
        chars_meta_rows.append(meta)

In [83]:
dataset = pd.read_json(f"{PWD}/data/datasets/{data_type}.jsonl", lines=True)

## query gpt

In [84]:
def classify_text(text, char):

    _, scene = text.split("\n", 1)

    # To get the tokeniser corresponding to a specific model in the OpenAI API:
    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

    prompt = f"""
Read the scenes below and then, according to the {label_mode.lower()} personality typology,   categorize {char}'s personality as {label1} for {label1_def} or "{label2}" for {label2_def} or 0 if you do not think there is enough relevant information. Response with only one word.

scenes:
    {scene}
    """

    prompt = enc.decode(enc.encode(prompt)[:4000])

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-3.5 Turbo engine
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1,  # Restrict output to only one token
        n=1,  # Generate only one response,
        temperature=0
    )

    # Extract the first choice (response)
    predicted_label = response.choices[0].message.content
    return prompt, predicted_label

In [85]:
def verify_output(pred):
    if pred.upper() not in [label1, label2, "0"]:
        raise Exception(f"Invalid response: '{pred}'")

In [86]:
chars = [char_id for char_id in chars_meta]
preds = {}
labels = {}

In [87]:
for row in tqdm.tqdm(dataset.iloc):
    if row.char_id in preds:
        continue

    char_name = chars_meta[row.char_id]["character_name"]
    scene = row.text

    # print(char_name)

    prompt, response = classify_text(scene, char_name)

    # print(prompt)
    # print()

    try:
        verify_output(response)
    except:
        print("invalid response")
        continue
    # print("Correct:", row.label)
    # print("Predicted:", response)
    if response == "0":
        continue
    preds[row.char_id] = response
    labels[row.char_id] = row.label


1it [00:00,  2.12it/s]

229it [01:44,  2.18it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
with open(f"{PWD}/data/gpt_preds/{data_type}_filtered-{len(preds)}.json", "w+") as fp:
    json.dump(preds, fp)

## evaluation

### overall metrics

In [ ]:
preds_len = len(preds)
# preds_len = 1897

with open(f"{PWD}/data/gpt_preds/{data_type}_filtered-{preds_len}.json", "r+") as fp:
    preds = json.load(fp)

In [88]:
for row in dataset.iloc:
    if row.char_id in preds:
        labels[row.char_id] = row.label

In [89]:
pred_array = [preds[char_id] for char_id in chars if char_id in preds]
label_array = [labels[char_id] for char_id in chars if char_id in preds]

In [90]:
print(len(pred_array), len(label_array))

160 160


In [91]:
f1 = f1_score(label_array, pred_array, labels=[label1, label2], average="macro")
prec = precision_score(label_array, pred_array, labels=[label1, label2], average="macro")
recall = recall_score(label_array, pred_array, labels=[label1, label2], average="macro")

print("overall:")
print("f1", f1)
print("precision", prec)
print("recall", recall)


overall:
f1 0.3712727104618372
precision 0.6019736842105263
recall 0.5196233581262858


### label 1 metrics

In [ ]:
f1 = f1_score(label_array, pred_array, labels=[label1, label2], pos_label=label1)
prec = precision_score(label_array, pred_array, labels=[label1, label2], pos_label=label1)
recall = recall_score(label_array, pred_array, labels=[label1, label2], pos_label=label1)

print(label1 + ":")
print("f1", f1)
print("precision", prec)
print("recall", recall)

S:
f1 0.6788990825688073
precision 0.548728813559322
recall 0.8900343642611683


### label 2 metrics

In [ ]:
f1 = f1_score(label_array, pred_array, labels=[label1, label2], pos_label=label2)
prec = precision_score(label_array, pred_array, labels=[label1, label2], pos_label=label2)
recall = recall_score(label_array, pred_array, labels=[label1, label2], pos_label=label2)

print(label2 + ":")
print("f1", f1)
print("precision", prec)
print("recall", recall)

R:
f1 0.2598187311178248
precision 0.5733333333333334
recall 0.16796875
